# PowerGrid Insights: Illuminating America's Outage Landscape

**Name(s)**: Eric Sun & Sunan Xu

**Website Link**: https://ericsun153.github.io/League_of_Legends_Match_Data_Analysis/

## Code

In [2]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

Read the csv file of our dataset.

In [3]:
df = pd.read_csv('2022_LoL_esports_match_data_from_OraclesElixir.csv')
df

D:\Anaconda\envs\dsc80\lib\site-packages\IPython\core\interactiveshell.py:3505: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149395,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DC,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149396,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DC,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149397,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DC,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149398,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DC,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning and EDA

Since the column of *url* is relatively unneccessary, and it contains a huge amount of NaN value, we can directly drop this column.

In [4]:
df = df.drop('url', axis=1)
df

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,participantid,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,1,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,2,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,3,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,4,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,5,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149395,9687-9687_game_5,partial,DC,2022,NaN,0,2022-12-27 12:43:43,5,12.23,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149396,9687-9687_game_5,partial,DC,2022,NaN,0,2022-12-27 12:43:43,5,12.23,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149397,9687-9687_game_5,partial,DC,2022,NaN,0,2022-12-27 12:43:43,5,12.23,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149398,9687-9687_game_5,partial,DC,2022,NaN,0,2022-12-27 12:43:43,5,12.23,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df[df['gameid'] == 'ESPORTSTMNT01_2690210']

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,participantid,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,1,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,2,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,3,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,4,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,5,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0
5,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,6,...,135.0,-391.0,-345.0,-14.0,0.0,1.0,0.0,0.0,1.0,0.0
6,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,7,...,89.0,-541.0,275.0,11.0,0.0,5.0,1.0,2.0,3.0,2.0
7,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,8,...,120.0,475.0,-153.0,-1.0,3.0,3.0,2.0,0.0,3.0,0.0
8,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,9,...,115.0,793.0,1343.0,34.0,3.0,3.0,0.0,2.0,1.0,2.0
9,ESPORTSTMNT01_2690210,complete,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,10,...,28.0,-443.0,497.0,-7.0,0.0,6.0,2.0,1.0,2.0,2.0


### Assessment of Missingness

In [ ]:
# TODO

### Hypothesis Testing

In [ ]:
# TODO